In [84]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset

import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F

import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output

import jieba.posseg as pseg

jieba.load_userdict(os.path.join('data', 'dict.txt.big'))
[jieba.add_word(i, freq=None, tag=None) for i in ['不支持','文林苑', '都更案','十八趴','證所稅','前瞻建設']]

[None, None, None, None, None, None]

In [85]:
top_num = 300

In [86]:
with open('data/stop_word_zhtw.txt') as file:
    data = file.read()
    
stop_words = data.split('\n')
stop_words += ['「', '」', '，', '\n', '）', '（', ')', '(']

In [87]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)

## sort the contents by index
keys, content_list = contents.keys(), contents.values

In [88]:
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

In [90]:
for test_id , text_q in enumerate(test_query):
    print(jieba.lcut(text_q))

['通姦', '在', '刑法', '上', '應該', '除罪化']
['應該', '取消', '機車', '強制', '二段式', '左轉', '(', '待轉', ')']
['支持', '博弈', '特區', '在', '台灣', '合法化']
['中華', '航空', '空服員', '罷工', '是', '合理', '的']
['性交易', '應該', '合法化']
['ECFA', '早', '收清單', '可', '（', '有', '）', '達到', '其', '預期', '成效']
['應該', '減免', '證所稅']
['贊成', '中油', '在', '觀塘', '興建', '第三', '天然氣', '接收站']
['支持', '中國', '學生', '納入', '健保']
['支持', '臺灣', '中小學', '（', '含', '高職', '、', '專科', '）', '服儀', '規定', '（', '含髮', '、', '襪', '、', '鞋', '）', '給予', '學生', '自主']
['不支持', '使用', '加密', '貨幣']
['不支持', '學雜費', '調漲']
['同意', '政府', '舉債', '發展', '前瞻建設', '計畫']
['支持', '電競', '列入', '體育競技']
['反對', '台鐵', '東移', '徵收案']
['支持', '陳前總統', '保外就醫']
['年金', '改革', '應', '取消', '或', '應', '調降', '軍公教', '月', '退之', '優存', '利率', '十八趴']
['同意', '動物', '實驗']
['油價', '應該', '凍漲', '或', '緩漲']
['反對', '旺旺', '中時', '併購', '中嘉']


In [91]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        sen = jieba.lcut(x, cut_all=False)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None

def psegcut(x):
    if x is not None:
        sen = pseg.lcut(x)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentence_arr = pool.map(jbcut,content_list)
pool.close()
pool.join()

In [9]:
# sentence_arr = np.load('arr.npy', allow_pickle=True)

## okapi model

In [93]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentence_arr)

In [94]:
stands = ['支持', '應該', '反對', '贊成', '不'] 

In [95]:
train_query = list(set(raw_training_data.Query))

y_train = []
y_index = {}
for i  in train_query:
    index = np.where(raw_training_data.Query==i)
    data = raw_training_data.iloc[index]
    y = dict(zip(data.News_Index,data.Relevance))
    y_idx = [ (int(idx.split('_')[1])-1, rel )  for idx, rel in zip(data.News_Index, data.Relevance)]
    y_train.append(y)
    y_index[i] = [y_idx]

In [96]:
common_query = set(train_query) & set(test_query)
common_query = list(common_query)
common = [ train_query.index(i) for i in common_query]

In [97]:
[ len(list(y_train[i].values())) for i in common]

[139, 230, 230, 40, 95]

In [98]:
[ np.sum(list(y_train[i].values())) / 300 for i in range(20)]

[0.6933333333333334,
 1.0533333333333332,
 0.21333333333333335,
 0.7066666666666667,
 1.3,
 2.7866666666666666,
 2.1933333333333334,
 0.77,
 2.296666666666667,
 1.4466666666666668,
 0.5766666666666667,
 2.6466666666666665,
 1.3733333333333333,
 1.35,
 0.2833333333333333,
 0.9433333333333334,
 1.3233333333333333,
 0.14,
 0.23,
 0.24333333333333335]

In [99]:
total_scores = list()
for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    
    top_query = np.argsort(scores)[::-1][:100]
    irrelevant_query = np.argsort(scores)[:100]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.5 * np.array(bm25Model.get_scores(text))

    irrelevant = []
    for query in irrelevant_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        irrelevant += top_words

    scores -= 0.5 * np.array(bm25Model.get_scores(irrelevant))
#     for idx, rel in (y_index[text_q][0]):
#         scores[idx] += np.mean(scores) * idx
    
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    validation_score = 0
    for a in ans:
        if a in y_train[test_id].keys():
            validation_score += y_train[test_id][a]
            
    total_scores.append(validation_score/top_num)
    print("{} / {} ".format(validation_score,(np.sum(list(y_train[test_id].values())))))

0 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
top query num 100
162 / 208 
1 核四應該啟用 ['核四', '應該', '啟用']
top query num 100
43 / 316 
2 反對無圍牆校園 ['反對', '圍牆', '校園']
top query num 100
59 / 64 
3 支持正名「臺灣」參與國際運動賽事 ['支持', '正名', '臺灣', '參與', '國際', '運動', '賽事']
top query num 100
154 / 212 
4 反對二代健保規定 ['反對', '二代', '健保', '規定']
top query num 100
231 / 390 
5 國際賽事會場內應該可以持中華民國國旗 ['國際賽', '事', '會場', '應該', '持', '中華民國', '國旗']
top query num 100
133 / 836 
6 堅決反對政府舉債發展前瞻建設計畫 ['堅決', '反對', '政府', '舉債', '發展', '前瞻建設', '計畫']
top query num 100
304 / 658 
7 另立專法保障同婚是正確的 ['另立', '專法', '保障', '同婚', '正確']
top query num 100
104 / 231 
8 贊同課綱微調 ['贊同', '課綱', '微調']
top query num 100
142 / 689 
9 贊成流浪動物零撲殺 ['贊成', '流浪', '動物', '零', '撲殺']
top query num 100
297 / 434 
10 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中職', '免', '學費', '補助', '適用', '對象', '增加', '不對']
top query num 100
114 / 173 
11 應該提高酒駕罰責以有效遏制酒駕 ['應該', '提高', '酒駕', '罰責', '有效', '遏制', '酒駕']
top query num 100
383 / 794 
12 臺灣應開放含瘦肉精(萊克多巴胺)之美國牛肉進口 ['臺灣', '應', '開放', '含', '瘦肉精', '

In [ ]:
print('Building dictionary...')
from gensim import corpora
data = sentence_arr
dictionary = corpora.Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]
corpora.MmCorpus.serialize('corpus.mm', corpus) 

from gensim import models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

corpora.MmCorpus.serialize('corpus.mm', corpus_tfidf) 
tfidf.save("my_model.tfidf")
tfidf = models.TfidfModel.load("my_model.tfidf")

In [122]:
print('Building LsiModel...')
corpus_tfidf = corpora.MmCorpus('corpus.mm')
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)

print('Building MatrixSimilarity...')
from gensim.similarities import MatrixSimilarity
index = MatrixSimilarity(lsi[corpus_tfidf])

index.save('deerwester.index')
index = MatrixSimilarity.load('deerwester.index')

Building LsiModel...


MemoryError: 

In [ ]:
total_scores = []
print('Testing...')
result = np.zeros((20,300)).astype('str')

j = 0
for test_id, doc in enumerate(train_query):
    tokens = jieba.lcut(doc)
    text = tokens
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    top_query = np.argsort(scores)[::-1][:100]
    irrelevant_query = np.argsort(scores)[:100]
    
    score = scores
    score = (score-min(score))/(max(score)-min(score))
    score *= 1.2
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    tmp = np.array(bm25Model.get_scores(text))
    tmp = (tmp-min(tmp))/(max(tmp)-min(tmp))
    scores += 0.2 * tmp
    
    irrelevant = []
    for query in irrelevant_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        irrelevant += top_words
        
    tmp = np.array(bm25Model.get_scores(irrelevant))
    tmp = (tmp-min(tmp))/(max(tmp)-min(tmp))
    scores -= 0.2 * tmp
    
    vec_bow = dictionary.doc2bow(tokens)
    vec_lsi = lsi[vec_bow]
    sims = index[vec_lsi]
    tmp = sims
    tmp = (tmp-min(tmp))/(max(tmp)-min(tmp))
    scores += tmp
        
    vec_bow = dictionary.doc2bow(text)
    vec_lsi = lsi[vec_bow]
    sims = index[vec_lsi]
    tmp = sims
    tmp = (tmp-min(tmp))/(max(tmp)-min(tmp))
    scores += 0.2 * tmp

    vec_bow = dictionary.doc2bow(irrelevant)
    vec_lsi = lsi[vec_bow]
    sims = index[vec_lsi]
    tmp = sims
    tmp = (tmp-min(tmp))/(max(tmp)-min(tmp))
    scores -= 0.2 * tmp

    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    validation_score = 0
    for a in ans:
        if a in y_train[test_id].keys():
            validation_score += y_train[test_id][a]
    
    total_scores.append(validation_score/top_num)
    print("{} / {} ".format(validation_score,(np.sum(list(y_train[test_id].values())))))

Testing...
0 拒絕公投通過門檻下修 ['反對', '旺旺', '中時', '併購', '中嘉']
top query num 100
105 / 208 
1 拒絕公投通過門檻下修 ['核四', '應該', '啟用']
top query num 100


In [140]:
np.mean(total_scores)

0.4445

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(contents)

In [71]:
from scipy.sparse import coo_matrix, vstack, csc_matrix
pfeature  = vstack([pos, neg])
plabel =  [1] * pos.shape[0] + [0] * neg.shape[0]

In [72]:
from random import shuffle
indices = np.arange(pfeature.shape[0]) #gets the number of rows 

combined = (list(zip(indices, plabel)))
random.shuffle(combined)
indices, plabel = zip(*combined)

pfeature = pfeature[list(indices)] 

In [74]:
trainX, testX, trainY, testY = train_test_split(pfeature, plabel, test_size=0.2)

In [80]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from xgboost import XGBClassifier
from keras.utils import to_categorical
from sklearn.svm import SVC

model =  SVC(C=0.5)
model.fit(trainX, trainY)

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [81]:
pred = model.predict(testX)

In [82]:
np.mean(testY == pred)

0.4975

In [53]:
total_scores = list()
for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    
    top_query = np.argsort(scores)[::-1][:1000]
    irrelevant_query = np.argsort(scores)[:1000]

    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))

    ## pseudo relevant feedback
    pos = X[top_query]
    neg = X[irrelevant_query]
    break
    plabel = [1] * pos.shape[0] + [0] * neg.shape[0]
    pfeature  = vstack([pos, neg])
    
    indices = np.arange(pfeature.shape[0])
    combined = (list(zip(indices, plabel)))
    random.shuffle(combined)
    indices, plabel = zip(*combined)
    pfeature = pfeature[list(indices)]
    
    model = GradientBoostingClassifier(n_estimators=100)
    model.fit(pfeature, plabel)
    scores += 0.5 * np.max(scores) * model.predict_proba(X)[:,1]

    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    validation_score = 0
    for a in ans:
        if a in y_train[test_id].keys():
            validation_score += y_train[test_id][a]
            
    total_scores.append(validation_score / top_num)
    print("{} / {} ".format(validation_score,(np.sum(list(y_train[test_id].values())))))

0 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
top query num 1000


$${\displaystyle k_{1}\in [1.2,2.0]}, {\displaystyle b=0.75}$$

In [ ]:
total_scores = list()
for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    
    top_query = np.argsort(scores)[::-1][:1000]
    irrelevant_query = np.argsort(scores)[:1000]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.5 * np.array(bm25Model.get_scores(text))

    irrelevant = []
    for query in irrelevant_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        irrelevant += top_words

    scores -= 0.5 * np.array(bm25Model.get_scores(irrelevant))
#     for idx, rel in (y_index[text_q][0]):
#         scores[idx] += np.mean(scores) * idx
    
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    validation_score = 0
    for a in ans:
        if a in y_train[test_id].keys():
            validation_score += y_train[test_id][a]
            
    total_scores.append(validation_score/top_num)
    print("{} / {} ".format(validation_score,(np.sum(list(y_train[test_id].values())))))

0 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
top query num 1000
168 / 208 
1 核四應該啟用 ['核四', '應該', '啟用']
top query num 1000


In [ ]:
np.mean(total_scores)

## Test the model

In [ ]:
top_num = 300
search_result = np.zeros((20,top_num))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:top_num]
    search_result[cnt] += keys
    
search_result = search_result.astype(np.int)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(top_num):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i] + 1

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))

In [ ]:
fname = 'simple27.csv'
assert fname not in (os.listdir('output/'))

In [ ]:
df.to_csv('output/' + fname, index=False)

# search_result[0]